# US Census Block Group FIPS codes

Download .shp file from https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_bg_500k.zip (via https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.2020.html)

In [59]:
!wget https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_bg_500k.zip

--2023-07-24 18:08:07--  https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_bg_500k.zip
Resolving www2.census.gov (www2.census.gov)... 2600:1400:d:492::208c, 2600:1400:d:4a5::208c, 23.4.233.168
Connecting to www2.census.gov (www2.census.gov)|2600:1400:d:492::208c|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘cb_2020_us_bg_500k.zip’

cb_2020_us_bg_500k.     [        <=>         ]  99.38M  6.49MB/s    in 15s     

2023-07-24 18:08:22 (6.54 MB/s) - ‘cb_2020_us_bg_500k.zip’ saved [104212096]



In [60]:
!unzip cb_2020_us_bg_500k.zip

Archive:  cb_2020_us_bg_500k.zip
  inflating: cb_2020_us_bg_500k.shp.ea.iso.xml  
  inflating: cb_2020_us_bg_500k.shp.iso.xml  
  inflating: cb_2020_us_bg_500k.shp  
  inflating: cb_2020_us_bg_500k.shx  
  inflating: cb_2020_us_bg_500k.dbf  
  inflating: cb_2020_us_bg_500k.prj  
 extracting: cb_2020_us_bg_500k.cpg  


In [61]:
!pip install shpinfo

In [68]:
!shpinfo cb_2020_us_bg_500k.shx


Fields
------

Field Name                         |Field Type    |  Field Length|Decimal Length
--------------------------------------------------------------------------------
STATEFP                            |Character     |             2|             0
COUNTYFP                           |Character     |             3|             0
TRACTCE                            |Character     |             6|             0
BLKGRPCE                           |Character     |             1|             0
AFFGEOID                           |Character     |            21|             0
GEOID                              |Character     |            12|             0
NAME                               |Character     |           100|             0
NAMELSAD                           |Character     |           100|             0
LSAD                               |Character     |             2|             0
ALAND                              |Number        |            14|             0
AWATER      

# Load the shx file with duckdb spatial extension

https://duckdb.org/2023/04/28/spatial.html

In [63]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## 

In [64]:
%%sql
INSTALL spatial;

,Success


In [65]:
%%sql
LOAD spatial;

,Success


In [69]:
%%sql
-- Create a table to hold the census block groups
CREATE TABLE census AS SELECT statefp, countyfp, tractce, blkgrpce, affgeoid, geoid, name, namelsad, lsad, aland, awater, ST_GeomFromWKB(wkb_geometry) AS geom 
FROM ST_Read('cb_2020_us_bg_500k.shx');

,Count
0,242298


In [70]:
%%sql
DESCRIBE census;

,column_name,column_type,null,key,default,extra
0,STATEFP,VARCHAR,YES,NaN,NaN,NaN
1,COUNTYFP,VARCHAR,YES,NaN,NaN,NaN
2,TRACTCE,VARCHAR,YES,NaN,NaN,NaN
3,BLKGRPCE,VARCHAR,YES,NaN,NaN,NaN
4,AFFGEOID,VARCHAR,YES,NaN,NaN,NaN
5,GEOID,VARCHAR,YES,NaN,NaN,NaN
6,NAME,VARCHAR,YES,NaN,NaN,NaN
7,NAMELSAD,VARCHAR,YES,NaN,NaN,NaN
8,LSAD,VARCHAR,YES,NaN,NaN,NaN
9,ALAND,BIGINT,YES,NaN,NaN,NaN


In [76]:
%%sql 
SELECT * FROM census LIMIT 10;

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geom
0,34,001,010401,1,1500000US340010104011,340010104011,1,Block Group 1,BG,22447299,38447,"[0, 2, 226, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
1,34,001,010403,2,1500000US340010104032,340010104032,2,Block Group 2,BG,5673534,0,"[0, 2, 162, 0, 188, 3, 68, 64, 2, 0, 0, 0, 1, ..."
2,45,083,021803,2,1500000US450830218032,450830218032,2,Block Group 2,BG,3164521,0,"[0, 2, 193, 0, 30, 55, 94, 192, 2, 0, 0, 0, 1,..."
3,45,083,022004,3,1500000US450830220043,450830220043,3,Block Group 3,BG,3059352,9817,"[0, 2, 145, 0, 214, 254, 67, 64, 2, 0, 0, 0, 1..."
4,45,083,021700,2,1500000US450830217002,450830217002,2,Block Group 2,BG,2222873,13205,"[0, 2, 193, 0, 59, 45, 94, 192, 2, 0, 0, 0, 1,..."
5,45,083,023101,3,1500000US450830231013,450830231013,3,Block Group 3,BG,2835493,71232,"[0, 2, 130, 0, 111, 243, 67, 64, 2, 0, 0, 0, 1..."
6,55,133,202400,3,1500000US551332024003,551332024003,3,Block Group 3,BG,1143632,0,"[0, 2, 65, 0, 125, 40, 94, 192, 2, 0, 0, 0, 1,..."
7,55,133,202001,2,1500000US551332020012,551332020012,2,Block Group 2,BG,12548322,915292,"[0, 2, 161, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
8,55,133,200803,3,1500000US551332008033,551332008033,3,Block Group 3,BG,1399329,7095,"[0, 2, 176, 0, 116, 235, 67, 64, 2, 0, 0, 0, 1..."
9,55,117,000400,1,1500000US551170004001,551170004001,1,Block Group 1,BG,2537326,6807,"[0, 2, 145, 0, 201, 50, 94, 192, 2, 0, 0, 0, 1..."


In [77]:
%%sql
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude, ST_Y(ST_Centroid(geom)) AS latitude FROM census LIMIT 10;

,GEOID,longitude,latitude
0,340010104011,-74.617847,-74.617847
1,340010104032,-74.558310,-74.558310
2,450830218032,-81.999324,-81.999324
3,450830220043,-81.853250,-81.853250
4,450830217002,-81.968185,-81.968185
5,450830231013,-82.117985,-82.117985
6,551332024003,-88.222655,-88.222655
7,551332020012,-88.286755,-88.286755
8,551332008033,-88.131004,-88.131004
9,551170004001,-87.746730,-87.746730


In [82]:
%%sql
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude FROM census LIMIT 10;

,GEOID,longitude
0,340010104011,-74.617847
1,340010104032,-74.558310
2,450830218032,-81.999324
3,450830220043,-81.853250
4,450830217002,-81.968185
5,450830231013,-82.117985
6,551332024003,-88.222655
7,551332020012,-88.286755
8,551332008033,-88.131004
9,551170004001,-87.746730


In [83]:
%%sql
SELECT GEOID, ST_Y(ST_Centroid(geom)) AS latitude FROM census LIMIT 10;

,GEOID,latitude
0,340010104011,39.526633
1,340010104032,39.464738
2,450830218032,35.012647
3,450830220043,34.907145
4,450830217002,34.959954
5,450830231013,34.916696
6,551332024003,42.996266
7,551332020012,42.906821
8,551332008033,43.096977
9,551170004001,43.760150


In [81]:
%%sql
WITH census_block_centroids AS (
  SELECT ST_transform(ST_Centroid(geom), 'EPSG:4269', 'EPSG:4326') AS centroid FROM census
),
centroids_lat_long AS (
  SELECT 
  ST_X(centroid) AS longitude, 
  ST_Y(centroid) AS latitude FROM census_block_centroids
)
SELECT * FROM centroids_lat_long LIMIT 10;

,longitude,latitude
0,-74.617847,-74.617847
1,-74.558310,-74.558310
2,-81.999324,-81.999324
3,-81.853250,-81.853250
4,-81.968185,-81.968185
5,-82.117985,-82.117985
6,-88.222655,-88.222655
7,-88.286755,-88.286755
8,-88.131004,-88.131004
9,-87.746730,-87.746730


In [103]:
%%sql
CREATE TABLE census_all AS SELECT statefp, countyfp, tractce, blkgrpce, affgeoid, geoid, name, namelsad, lsad, aland, awater, ST_GeomFromWKB(wkb_geometry) AS geom 
FROM ST_Read('/Users/me/Downloads/cb_2020_us_bg_500k/cb_2020_us_bg_500k.shx');

,Count
0,242298


In [105]:
%%sql 
CREATE TABLE census_all_long AS 
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude, statefp, countyfp, tractce, blkgrpce, affgeoid, name, namelsad, lsad, aland
FROM census_all;

,Count
0,242298


In [106]:
%%sql 
CREATE TABLE census_only_lat AS 
SELECT GEOID, ST_Y(ST_Centroid(geom)) AS latitude 
FROM census;

,Count
0,242298


In [109]:
%%sql 
CREATE TABLE census_all_latlong AS
SELECT 
  census_all_long.*,
  census_only_lat.latitude,
FROM census_all_long
JOIN census_only_lat ON census_only_lat.GEOID = census_all_long.GEOID;

,Count
0,242298


In [110]:
%%sql
COPY (
  SELECT * FROM census_all_latlong
) TO '/Users/me/Downloads/census_block_groups_latitude_longitude.parquet' (COMPRESSION ZSTD);

,Count
0,242298


In [111]:
import polars as pl

In [112]:
pl.read_parquet('/Users/me/Downloads/census_block_groups_latitude_longitude.parquet')

GEOID,longitude,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,NAME,NAMELSAD,LSAD,ALAND,latitude
str,f64,str,str,str,str,str,str,str,str,i64,f64
"""060590422062""",-117.649586,"""06""","""059""","""042206""","""2""","""1500000US06059…","""2""","""Block Group 2""","""BG""",989367,33.451279
"""060376025051""",-118.341669,"""06""","""037""","""602505""","""1""","""1500000US06037…","""1""","""Block Group 1""","""BG""",161423,33.90737
"""060590421092""",-117.598564,"""06""","""059""","""042109""","""2""","""1500000US06059…","""2""","""Block Group 2""","""BG""",547167,33.442898
"""360810113002""",-73.908686,"""36""","""081""","""011300""","""2""","""1500000US36081…","""2""","""Block Group 2""","""BG""",57260,40.776412
"""360290059004""",-78.901426,"""36""","""029""","""005900""","""4""","""1500000US36029…","""4""","""Block Group 4""","""BG""",122062,42.937805
"""360050442003""",-73.8454,"""36""","""005""","""044200""","""3""","""1500000US36005…","""3""","""Block Group 3""","""BG""",46978,40.904587
"""060855033123""",-121.798139,"""06""","""085""","""503312""","""3""","""1500000US06085…","""3""","""Block Group 3""","""BG""",211554,37.35352
"""360290024002""",-78.806804,"""36""","""029""","""002400""","""2""","""1500000US36029…","""2""","""Block Group 2""","""BG""",163523,42.894579
"""060590018014""",-117.979217,"""06""","""059""","""001801""","""4""","""1500000US06059…","""4""","""Block Group 4""","""BG""",359476,33.860834
